In [3]:
#Position-value dependent

#Unpack necessities
from datetime import datetime
from dateutil.tz import tzlocal
import pandas as pd
import numpy as np
import my_config
import schedule
import time
import ccxt

import warnings
warnings.filterwarnings('ignore')

#Elimanates time/order issues, not really sure why this works...
ccxt.binanceus({ 'options':{ 'adjustForTimeDifference':True}})

exchange = ccxt.binanceus({
"apiKey": my_config.BINANCE_KEY,
"secret": my_config.BINANCE_SECRET,
'enableRateLimit': True})

#Super trend formula.
def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['previous_close'])
    data['low-pc'] = abs(data['low'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean()

    return atr

def supertrend(df, period=7, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

###########################################################

#####Parameters for POSITION-VALUE-DEPENDENT TRADE AMOUNT##
#ticker = input("Insert ticker (XXX/USD): ")+'/USD'

#bal = pd.DataFrame(exchange.fetch_balance()['info']['balances'])
#bal['free'] = pd.to_numeric(bal['free'])
#bal = bal[bal.free!=0].drop(columns='locked').reset_index(drop=True)
#bal = bal[bal['asset']==ticker[:4].replace('/','')].reset_index(drop=True).free[0]
#in_position = int(bal)>0

#Percentage of dedicated trading value
#perc = int(input("Percentage of tradeable amount: "))/100
#order_size = int(bal*perc)
############################################################


####Parameters for HARD-VALUE TRADE AMOUNT####
#in_position = False
ticker = 'HBAR/USD'
trade_amount = 20
#bar = exchange.fetch_ohlcv(f'{ticker}', timeframe='1m', limit=5)
#order_size = int(trade_amount/bar[4][1]-(.05*(trade_amount/bar[4][1]))) #Trades $1000.

#Parameters for POSITION-DEPENDENT TRADE AMOUNT
ticker = "VTHO/USD"#input("Insert ticker (XXX/USD): ")+'/USD'

#Get position_val
bal = pd.DataFrame(exchange.fetch_balance()['info']['balances'])
bal['free'] = pd.to_numeric(bal['free'])
bal = bal[bal.free!=0].drop(columns='locked').reset_index(drop=True)
bal = bal[bal['asset']==ticker[:4].replace('/','')].reset_index(drop=True).free[0]
in_position = int(bal)>0

#Percentage of dedicated trading value
#perc = int(input("Percentage of tradeable amount: "))/100
#order_size = int(bal*perc)

bar = exchange.fetch_ohlcv(f'{ticker}', timeframe='1m', limit=5)
order_size = int(trade_amount/bar[4][1]-(.05*(trade_amount/bar[4][1]))) #Trades $1000.

#Decision maker.
def check_buy_sell_signals(df):
    global in_position,order_size,ticker
    print("Analyzing",ticker,"data... \nIn_position:",in_position,'\n')
    print(df.tail(3)[['timestamp','open','in_uptrend']])
    last_row_index = len(df.index) - 1
    previous_row_index = last_row_index - 1
    
    if not df['in_uptrend'][previous_row_index] and df['in_uptrend'][last_row_index]:
        print("Changed to uptrend - Buy")
        if not in_position:
            order = exchange.create_market_buy_order(f'{ticker}',order_size)
            print('Status:'+order['info']['status'],
                  'Price:'+order['trades'][0]['info']['price'],
                  'Quantity:'+order['info']['executedQty'],
                  'Type:'+order['info']['side'])  
            buys.append({'Date':datetime.now().isoformat(),
                            'Status':order['info']['status'],
                            'Price':order['trades'][0]['info']['price'],
                            'Quantity':order['info']['executedQty'],
                            'Type':order['info']['side']})   
            in_position = True
        else:
            print("Currently in position, no task.")
            print("Previous purchase price:",buys[len(buys)-1]['Price'])
    
    if df['in_uptrend'][previous_row_index] and not df['in_uptrend'][last_row_index]:
        bar = exchange.fetch_ohlcv(f'{ticker}', timeframe=timeframe, limit=5)
        price = bar[-1][1]
        print("Changed to downtrend.")
        try:
            if in_position and price > buys[len(buys)-1]['Price']:
                print("Purchase price < current_market_price - Sell.")
                order = exchange.create_market_sell_order(f'{ticker}',order_size)
                print('Status:'+order['info']['status'],
                      'Price:'+order['trades'][0]['info']['price'],
                      'Quantity:'+order['info']['executedQty'],
                      'Type:'+order['info']['side'])
                sells.append({'Date':datetime.now().isoformat(),
                              'Status':order['info']['status'],
                              'Price':order['trades'][0]['info']['price'],
                              'Quantity':order['info']['executedQty'],
                              'Type':order['info']['side']})
                in_position = False
            else:
                print("No selling position, no task.")
        except:
            if in_position:
                
                print("No purchase price data. - Sell.")
                order = exchange.create_market_sell_order(f'{ticker}',order_size)
                print('Status:'+order['info']['status'],
                      'Price:'+order['trades'][0]['info']['price'],
                      'Quantity:'+order['info']['executedQty'],
                      'Type:'+order['info']['side'])
                sells.append({'Date':datetime.now().isoformat(),
                              'Status':order['info']['status'],
                              'Price':order['trades'][0]['info']['price'],
                              'Quantity':order['info']['executedQty'],
                              'Type':order['info']['side']}) 
                in_position = False
            else:
                print("No selling position, no task.")

In [ ]:
#Run
def run_bot():
    print(f"\n\nFetching new bars for {datetime.now().isoformat()}")
    bars = exchange.fetch_ohlcv(f'{ticker}', timeframe='1m', limit=100)
    df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms') 
    supertrend_data = supertrend(df)
    check_buy_sell_signals(supertrend_data)
    print()
schedule.every(1).minutes.do(run_bot)
while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
pd.DataFrame(exchange.fetch_ohlcv("DOGE/USD", timeframe='1m', limit=5), columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])['open'][1]



In [21]:
df = pd.DataFrame(exchange.fetch_ohlcv("DOGE/USD", timeframe='1m', limit=5), columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
prev_order = []

def check_buy_sell_signals(df):
    prev_order.append({"Date":datetime.now(tzlocal()).isoformat(),
                       "Price":df['open'][1]})
    print(datetime.now(tzlocal()).isoformat(), df['open'][1])


def run_bot():
    print(f"\n\nFetching new bars for {datetime.now(tzlocal()).isoformat()}")
    check_buy_sell_signals(df)
    print("Previous purchase price:",prev_order[len(prev_order)-1]['Price'])
schedule.every(10).seconds.do(run_bot)
while True:
    schedule.run_pending()
    time.sleep(1)



Fetching new bars for 2021-09-14T16:00:49.723894-07:00
2021-09-14T16:00:49.724763-07:00 0.2404
Previous purchase price: 0.2404


Fetching new bars for 2021-09-14T16:00:49.725079-07:00
2021-09-14T16:00:49.725347-07:00 0.2404
Previous purchase price: 0.2404


Fetching new bars for 2021-09-14T16:00:49.725608-07:00
2021-09-14T16:00:49.725869-07:00 0.2404


Fetching new bars for 2021-09-14T16:00:49.726054-07:00
2021-09-14T16:00:49.726303-07:00 0.2404


Fetching new bars for 2021-09-14T16:00:49.726481-07:00
2021-09-14T16:00:49.726728-07:00 0.2404


KeyboardInterrupt: 

In [22]:
prev_order

[{'Date': '2021-09-14T16:00:49.724302-07:00', 'Price': 0.2404},
 {'Date': '2021-09-14T16:00:49.725221-07:00', 'Price': 0.2404},
 {'Date': '2021-09-14T16:00:49.725744-07:00', 'Price': 0.2404},
 {'Date': '2021-09-14T16:00:49.726185-07:00', 'Price': 0.2404},
 {'Date': '2021-09-14T16:00:49.726610-07:00', 'Price': 0.2404}]

In [16]:
order[len(order)-1]['Price']

0.2396

In [ ]:
bars = exchange.fetch_ohlcv('VTHO/USD', timeframe='1m', limit=5)
int(10/bars[4][1]-(.05*(10/bars[4][1])))

In [ ]:
exchange.create_market_buy_order('VTHO/USD',1100)

In [ ]:
order = {'info': {'symbol': 'VTHOUSD',
  'orderId': '21356239',
  'orderListId': '-1',
  'clientOrderId': 'x-R4BD3S82c4c4b3abd4e71dc009d278',
  'transactTime': '1631518396580',
  'price': '0.0000',
  'origQty': '1100.00000000',
  'executedQty': '1100.00000000',
  'cummulativeQuoteQty': '10.4500',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'fills': [{'price': '0.0095',
    'qty': '1100.00000000',
    'commission': '0.00001992',
    'commissionAsset': 'BNB',
    'tradeId': '4227218'}]},
 'id': '21356239',
 'clientOrderId': 'x-R4BD3S82c4c4b3abd4e71dc009d278',
 'timestamp': 1631518396580,
 'datetime': '2021-09-13T07:33:16.580Z',
 'lastTradeTimestamp': None,
 'symbol': 'VTHO/USD',
 'type': 'market',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 0.0095,
 'stopPrice': None,
 'amount': 1100.0,
 'cost': 10.45,
 'average': 0.0095,
 'filled': 1100.0,
 'remaining': 0.0,
 'status': 'closed',
 'fee': {'cost': 1.992e-05, 'currency': 'BNB'},
 'trades': [{'info': {'price': '0.0095',
    'qty': '1100.00000000',
    'commission': '0.00001992',
    'commissionAsset': 'BNB',
    'tradeId': '4227218'},
   'timestamp': None,
   'datetime': None,
   'symbol': 'VTHO/USD',
   'id': None,
   'order': None,
   'type': None,
   'side': None,
   'takerOrMaker': None,
   'price': 0.0095,
   'amount': 1100.0,
   'cost': 10.45,
   'fee': {'cost': 1.992e-05, 'currency': 'BNB'}}],
 'fees': [{'cost': 1.992e-05, 'currency': 'BNB'}]}

In [ ]:
#Order details. printable.
order['info']['symbol'],order['info']['orderId'], order['info']['status'], order['trades'][0]['info']['price'], order['info']['executedQty']

In [ ]:
order['trades'][0]['info']['price']

# Set condition(s) for variable: in_position

In [ ]:
df = pd.DataFrame(exchange.fetch_balance()['info']['balances'])
df['free'] = pd.to_numeric(df['free'])
df['locked'] = pd.to_numeric(df['locked'])
df = df[df.free!=0].drop(columns='locked').reset_index(drop=True)

In [ ]:
df[df['asset']=='HBAR'].free

In [ ]:
#Parameters for POSITION-DEPENDENT TRADE AMOUNT
ticker = input("Insert ticker (XXX/USD): ")+'/USD'

#Get position_val
bal = pd.DataFrame(exchange.fetch_balance()['info']['balances'])
bal['free'] = pd.to_numeric(bal['free'])
bal = bal[bal.free!=0].drop(columns='locked').reset_index(drop=True)
bal = bal[bal['asset']==ticker[:4].replace('/','')].reset_index(drop=True).free[0]
in_position = int(bal)>0
in_position

In [ ]:
bal[bal['asset']==ticker[:4].replace('/','')].reset_index(drop=True).free[0]

In [ ]:
ticker[:4]

In [ ]:
"""API Request Limit

* 1200 requests per minute
* 10 orders per second
* 100,000 orders per 24hrs

"""

# Sell ONLY IF current_price > previous_purchase_price

In [31]:
#Unpack necessities
from dateutil.tz import tzlocal
from datetime import datetime
import pandas as pd
import numpy as np
import my_config
import schedule
import time
import ccxt

import warnings
warnings.filterwarnings('ignore')

#Elimanates time/order issues, not really sure why this works...
ccxt.binanceus({ 'options':{ 'adjustForTimeDifference':True}})

exchange = ccxt.binanceus({
"apiKey": my_config.BINANCE_KEY,
"secret": my_config.BINANCE_SECRET,
'enableRateLimit': True})

#Super trend formula.
def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['previous_close'])
    data['low-pc'] = abs(data['low'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean()

    return atr

def supertrend(df, period=7, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

####Parameters for INDEPENDENT HARD-VALUE TRADE AMOUNT####
ticker = 'DOGE/USD'
trade_amount = 11
timeframe = "3m" #str(input("Candlestick timeframe: (1m,3m,15m,1d) "))

bal = pd.DataFrame(exchange.fetch_balance()['info']['balances'])
bal['free'] = pd.to_numeric(bal['free'])
bal = bal[bal.free!=0].drop(columns='locked').reset_index(drop=True)
bal = bal[bal['asset']==ticker[:4].replace('/','')].reset_index(drop=True).free[0]
in_position = int(bal)>0

bar = exchange.fetch_ohlcv(f'{ticker}', timeframe=timeframe, limit=5) #price = bar[-1][1]
order_size = int(trade_amount/bar[4][1]-(.05*(trade_amount/bar[4][1])))

#Collects previous orders.
buys,sells = [],[]

#Decision maker.
def check_buy_sell_signals(df):
    global in_position,order_size,ticker,timeframe,trade_amount,bal
    
    print("Analyzing",ticker,"data... ")
    print(df.tail(3)[['timestamp','open','in_uptrend']])
    
    last_row_index = len(df.index) - 1
    previous_row_index = last_row_index - 1
    
    if not df['in_uptrend'][previous_row_index] and df['in_uptrend'][last_row_index]:
        print("Changed to uptrend - Buy")
        if not in_position:
            order = exchange.create_market_buy_order(f'{ticker}',order_size)
            print('Status:'+order['info']['status'],
                  'Price:'+order['trades'][0]['info']['price'],
                  'Quantity:'+order['info']['executedQty'],
                  'Type:'+order['info']['side'])  
            buys.append({'Status':order['info']['status'],
                  'Price':order['trades'][0]['info']['price'],
                  'Quantity':order['info']['executedQty'],
                  'Type':order['info']['side']})  
            in_position = True
        else:
            print("Currently in position, no task.")
            print("Previous purchase price:",buys[len(buys)-1]['Price'])
    
    if df['in_uptrend'][previous_row_index] and not df['in_uptrend'][last_row_index]:
        bar = exchange.fetch_ohlcv(f'{ticker}', timeframe=timeframe, limit=5)
        price = bar[-1][1]
        if in_position and price > buys[len(buys)-1]['Price']: 
            print("Changed to downtrend - Sell")
            order = exchange.create_market_sell_order(f'{ticker}',order_size)
            print('Status:'+order['info']['status'],
                  'Price:'+order['trades'][0]['info']['price'],
                  'Quantity:'+order['info']['executedQty'],
                  'Type:'+order['info']['side'])
            sells.append({'Status':order['info']['status'],
                  'Price':order['trades'][0]['info']['price'],
                  'Quantity':order['info']['executedQty'],
                  'Type':order['info']['side']}) 
            in_position = False
        else:
            print("No selling position, no task.")

#Run le bot.
def run_bot():
    print(f"\n\nFetching new bars for {datetime.now(tzlocal()).isoformat()}")
    print("In position:", in_position,"; Balance: $",bal*bar[-1][1],"; Timeframe: ",timeframe,"; Trade amount: ",trade_amount)
    bars = exchange.fetch_ohlcv(f'{ticker}', timeframe=timeframe, limit=100)
    df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    supertrend_data = supertrend(df)
    check_buy_sell_signals(supertrend_data)
    schedule.every(3).seconds.do(run_bot)
while True:
    schedule.run_pending()
    time.sleep(1)



Fetching new bars for 2021-09-14T16:27:10.591556-07:00
Analyzing DOGE/USD data... 
             timestamp    open  in_uptrend
96 2021-09-14 23:15:00  0.2401        True
97 2021-09-14 23:18:00  0.2399        True
98 2021-09-14 23:21:00  0.2399        True
Previous purchase price: 0.2404


Fetching new bars for 2021-09-14T16:27:10.595343-07:00
Analyzing DOGE/USD data... 
             timestamp    open  in_uptrend
96 2021-09-14 23:15:00  0.2401        True
97 2021-09-14 23:18:00  0.2399        True
98 2021-09-14 23:21:00  0.2399        True
Previous purchase price: 0.2404


Fetching new bars for 2021-09-14T16:27:10.600941-07:00
Analyzing DOGE/USD data... 
             timestamp    open  in_uptrend
96 2021-09-14 23:15:00  0.2401        True
97 2021-09-14 23:18:00  0.2399        True
98 2021-09-14 23:21:00  0.2399        True
Previous purchase price: 0.2404


Fetching new bars for 2021-09-14T16:27:10.608458-07:00
Analyzing DOGE/USD data... 
             timestamp    open  in_uptrend
96 20

KeyboardInterrupt: 

In [32]:
buys

[]

In [28]:
bars = exchange.fetch_ohlcv(f'{ticker}', timeframe=timeframe, limit=100)
df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
supertrend_data = supertrend(df)
check_buy_sell_signals(supertrend_data)

Analyzing DOGE/USD data... 


In [29]:

supertrend_data

,timestamp,open,high,low,close,volume,previous_close,high-low,high-pc,low-pc,tr,atr,upperband,lowerband,in_uptrend
0,2021-09-14 18:27:00,0.2383,0.2385,0.2383,0.2383,62171.0,NaN,0.0002,NaN,NaN,0.0002,NaN,NaN,NaN,True
1,2021-09-14 18:30:00,0.2383,0.2383,0.2379,0.2381,87985.0,0.2383,0.0004,0.0000,0.0004,0.0004,NaN,NaN,NaN,True
2,2021-09-14 18:33:00,0.2381,0.2383,0.2378,0.2383,180611.0,0.2381,0.0005,0.0002,0.0003,0.0005,NaN,NaN,NaN,True
3,2021-09-14 18:36:00,0.2382,0.2383,0.2379,0.2379,190730.0,0.2383,0.0004,0.0000,0.0004,0.0004,NaN,NaN,NaN,True
4,2021-09-14 18:39:00,0.2379,0.2381,0.2377,0.2381,128343.0,0.2379,0.0004,0.0002,0.0002,0.0004,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2021-09-14 23:09:00,0.2405,0.2407,0.2403,0.2403,80594.0,0.2407,0.0004,0.0000,0.0004,0.0004,0.000486,0.241957,0.239286,True
95,2021-09-14 23:12:00,0.2403,0.2403,0.2399,0.2403,155763.0,0.2403,0.0004,0.0000,0.0004,0.0004,0.000457,0.241471,0.239286,True
96,2021-09-14 23:15:00,0.2401,0.2401,0.2395,0.2399,260076.0,0.2403,0.0006,0.0002,0.0008,0.0008,0.000514,0.241343,0.239286,True
97,2021-09-14 23:18:00,0.2399,0.2401,0.2398,0.2398,62596.0,0.2399,0.0003,0.0002,0.0001,0.0003,0.000457,0.241321,0.239286,True


In [ ]:
#PSUEDO CODE
#Collection of previous orders.
buys,sells = [],[]

#Decision maker.
def check_buy_sell_signals(df):
    if previous_row_downtrend and last_row_index_uptrend:
        #BUY
        if not in_position:
            order = exchange.create_market_buy_order() 
            
            buys.append(buy_order_info)  
            
            in_position = True
        else:
            print("Currently in position, no task.")
    
    if previous_row_uptrend and last_row_downtrend:
        bar = exchange.fetch_data
        price = bar[-1][1]
        #SELL
        if in_position and price > buys[len(buys)-1]['Price']:
            order = exchange.create_market_sell_order()
            
            sells.append(sell_order_info) 
            
            in_position = False
        else:
            print("No sell.")

#Iterative
def run_bot():
    bars = exchange.fetch_data
    check_buy_sell_signals(algorithm(bars))
schedule.every(30).minutes.do(run_bot)
while True:
    schedule.run_pending()
    time.sleep(1)

In [33]:
import csv

In [34]:
buys.to_csv('buy_orders.csv',index=False)


[]

In [36]:
ticker = 'DOGE/USD'
trade_amount = 800
timeframe = "5m" #str(input("Candlestick timeframe: (1m,3m,5m,10m,15m,30m,1d) "))

bal = pd.DataFrame(exchange.fetch_balance()['info']['balances'])
bal['free'] = pd.to_numeric(bal['free'])
bal = bal[bal.free!=0].drop(columns='locked').reset_index(drop=True)
bal = bal[bal['asset']==ticker[:4].replace('/','')].reset_index(drop=True).free[0]

bar = exchange.fetch_ohlcv(f'{ticker}', timeframe=timeframe, limit=2)
order_size# = int(trade_amount/bar[4][1]-(.05*(trade_amount/bar[4][1])))
in_position = int(bal*bar[-1][1])

In [37]:
order_size

43

In [63]:
exchange.fetch_ohlcv(f'{ticker}', timeframe="5m", limit=5)[0][1]

0.2411

In [60]:
bar[0][3]

0.2395

In [49]:
3 > 3+1

False

In [58]:
float(8.888)+float(8.888)*0.016

9.030208

In [59]:
9.03/8.888

1.015976597659766